<a href="https://colab.research.google.com/github/nitin649/QuestionAnswering_Using_T5/blob/main/QuestionAnsweringT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet transformers==4.1.1
!pip install --quiet tokenizers==0.9.4

!pip install --quiet torchtext==0.10.0 
#!pip install --quiet pytorch-lightning==1.1.1
!pip install --quiet sentencepiece

     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 2.9 MB 40.9 MB/s 
     |████████████████████████████████| 895 kB 29.8 MB/s 
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:

!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install --quiet --upgrade pytorch-lightning
#torch.__version__

     |████████████████████████████████| 925 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
     |████████████████████████████████| 282 kB 57.1 MB/s 
     |████████████████████████████████| 829 kB 50.1 MB/s 
     |████████████████████████████████| 125 kB 47.6 MB/s 
     |████████████████████████████████| 1.3 MB 43.5 MB/s 
     |████████████████████████████████| 160 kB 72.4 MB/s 
     |████████████████████████████████| 271 kB 71.6 MB/s 


In [ ]:
import json
import pandas as pd
import numpy as np
import random
import logging
import glob 
import os
import re
import argparse
import time
from string import punctuation
import torch
from torch.utils.data import DataLoader,Dataset
import textwrap
from sklearn.model_selection import train_test_split
from termcolor import colored
from pathlib import Path
import pytorch_lightning as pl
from transformers import AdamW,T5ForConditionalGeneration,T5Tokenizer,get_linear_schedule_with_warmup

In [ ]:
pl.seed_everything(42)

Global seed set to 42


42

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#data

In [ ]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/QA.zip"

replace BioASQ/4B1_golden.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
with Path(r"/content/BioASQ/BioASQ-train-factoid-4b.json").open() as file:
  data=json.load(file)

In [ ]:
data.keys()

dict_keys(['data', 'version'])

In [ ]:
data['data'][0].keys()

dict_keys(['paragraphs', 'title'])

In [ ]:
len(data['data'][0]['paragraphs'])

3266

In [ ]:
questions=data['data'][0]['paragraphs']
questions[0]

{'context': 'Balanced t(11;15)(q23;q15) in a TP53+/+ breast cancer patient from a Li-Fraumeni syndrome family. Li-Fraumeni Syndrome (LFS) is characterized by early-onset carcinogenesis involving multiple tumor types and shows autosomal dominant inheritance. Approximately 70% of LFS cases are due to germline mutations in the TP53 gene on chromosome 17p13.1. Mutations have also been found in the CHEK2 gene on chromosome 22q11, and others have been mapped to chromosome 11q23. While characterizing an LFS family with a documented defect in TP53, we found one family member who developed bilateral breast cancer at age 37 yet was homozygous for wild-type TP53. Her mother also developed early-onset primary bilateral breast cancer, and a sister had unilateral breast cancer and a soft tissue sarcoma. Cytogenetic analysis using fluorescence in situ hybridization of a primary skin fibroblast cell line revealed that the patient had a novel balanced reciprocal translocation between the long arms of c

In [ ]:
#creating dataframe using this data
def extract_ques_ans(path):
  with Path(path).open() as json_file:
    data=json.load(json_file)
  questions=data['data'][0]['paragraphs']
  data_rows=[]
  for question in questions:
    context=question['context']
    for ques_and_ans in question['qas']:
      question=ques_and_ans['question']
      answers=ques_and_ans['answers']

      for answer in answers:
        answer_text=answer['text']
        answer_start=answer['answer_start']
        answer_end=answer_start + len(answer_text)

        data_rows.append(
            {
                'question':question,
                'context':context,
                'answer_text':answer_text,
                'answer_start':answer_start,
                'answer_end':answer_end
            }
        )
  return pd.DataFrame(data_rows)

In [ ]:
extract_ques_ans(r"/content/BioASQ/BioASQ-train-factoid-4b.json").head()

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [ ]:
all_files_path=sorted(list(Path(r"/content/BioASQ/").glob("BioASQ-train-*")))
all_files_path

[PosixPath('/content/BioASQ/BioASQ-train-factoid-4b.json'),
 PosixPath('/content/BioASQ/BioASQ-train-factoid-5b.json'),
 PosixPath('/content/BioASQ/BioASQ-train-factoid-6b.json')]

In [ ]:
#appending all the data
dfs=[]
for path in all_files_path:
  df=extract_ques_ans(path)
  dfs.append(df)

df=pd.concat(dfs)

In [ ]:
df.shape

(12988, 5)

In [ ]:
#dropping the data which having same context
df=df.drop_duplicates(subset=['context']).reset_index(drop=True)

In [ ]:
len(df.question.unique())

441

In [ ]:
len(df.context.unique())

2582

In [ ]:
df.shape

(2582, 5)

In [ ]:
#tokenization

In [ ]:
!pip install --quiet sentencepiece==0.1.94
model_name='t5-base'
tokenizer=T5Tokenizer.from_pretrained(model_name)

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [ ]:
tokenizer('hello world')

{'input_ids': [21820, 296, 1], 'attention_mask': [1, 1, 1]}

In [ ]:
sample=tokenizer('what is your name?','my name is jhon')

In [ ]:
sample.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
print([tokenizer.decode(i) for i in sample['input_ids']])
" ".join([tokenizer.decode(i) for i in sample['input_ids']])

['what', 'is', 'your', 'name', '?', '</s>', 'my', 'name', 'is', '', 'j', 'h', 'on', '</s>']


'what is your name ? </s> my name is  j h on </s>'

In [ ]:
tokenizer.special_tokens_map

{'additional_special_tokens': "['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53>', '<extra_id_54>', '<extra_id_55>', '<extra_id_56>', '<extra_i

In [ ]:
tokenizer.eos_token,tokenizer.eos_token_id

('</s>', 1)

In [ ]:
tokenizer.decode(sample['input_ids'])

'what is your name?</s> my name is jhon</s>'

In [ ]:
sample_question=df.iloc[240]
encoding=tokenizer(
    sample_question['question'],
    sample_question['context'],
    max_length=396,
    padding='max_length',
    truncation='only_second',#truncate only the context not the question,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors='pt'
)

In [ ]:
print(encoding['input_ids'])
print(encoding['input_ids'].flatten().shape)

tensor([[  363,    19,     8, 16115,  1451,    13,     8, 12991,  1050,    18,
           308,     9,  6961,  1647,    18, 27189,   106, 12398,     5,     1,
         14298,     3,  6015,    23,  9475,    11,  5069,     3,     7,   994,
         10138,   663,    13, 24387,     3,   107, 11658,    17, 29006,    41,
           308,    63,  1050,    18,   308,     9,  6961,  1647,    18, 27189,
           106, 27956,   137,  1875,  2252,  6207,  7469,    13, 24387,     3,
           107, 11658,    17, 29006,    41,   308,    63,  1050,    18,   308,
             9,  6961,  1647,    18, 27189,   106, 27956,    61,    33,   168,
           801,     6,   132,    19,   150, 20036,   810,    81,     8,  7285,
            11,     8,  4161,   596,    16,    48, 12398,     5, 14170,  1383,
            16,  2208,  1221,    41,   526,   152,  9742,   850,    61,    28,
         24387,     3,   107, 11658,    17, 29006,   130, 29825,   120,  9112,
             5, 19636,  6808,  1221, 13649,  9285,  

In [ ]:
tokenizer.decode(encoding['input_ids'].squeeze())

'What is the characteristic feature of the Dyke-Davidoff-Masson syndrome.</s> Left hemisphere and male sex dominance of cerebral hemiatrophy (Dyke-Davidoff-Masson Syndrome). Although radiological findings of cerebral hemiatrophy (Dyke-Davidoff-Masson Syndrome) are well known, there is no systematic study about the gender and the affected side in this syndrome. Brain images in 26 patients (mean aged 11) with cerebral hemiatrophy were retrospectively reviewed. Nineteen patients (73.5%) were male and seven patients (26.5%) were female. Left hemisphere involvement was seen in 18 patients (69.2%) and right hemisphere involvement was seen in eight patients (30.8%). We conclude that male gender and left side involvement are frequent in cerebral hemiatrophy disease.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

In [ ]:
answer_encoding=tokenizer(
    sample_question['answer_text'],
    max_length=32,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors='pt'

)

In [ ]:
tokenizer.decode(answer_encoding['input_ids'].squeeze())

'cerebral hemiatrophy</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [ ]:
labels=answer_encoding['input_ids']
labels

tensor([[24387,     3,   107, 11658,    17, 29006,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

In [ ]:
#as per the huggingface t5 documentation pad values should be -100 are ignored (masked) and model take padded values this way only
labels[labels==0]=-100

In [ ]:
labels

tensor([[24387,     3,   107, 11658,    17, 29006,     1,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100]])

In [ ]:
#creating dataset for training
class BioASQDataset(Dataset):
  def __init__(self,dataframe,tokenizer,source_max_len=396,tar_max_len=32):
    super().__init__()
    self.tokenizer=tokenizer
    self.source_max_len=source_max_len
    self.tar_max_len=tar_max_len
    self.dataframe=dataframe
    
  def __len__(self):
    return len(self.dataframe)
  def __getitem__(self,index):
    data_row=self.dataframe.iloc[index]

    source_encoding=tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_len,
      padding='max_length',
      truncation='only_second',#truncate only the context not the question,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors='pt'
      )
    target_encoding=tokenizer(
        data_row['answer_text'],
        max_length=self.tar_max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'

    )
    labels=target_encoding['input_ids']
    labels[labels==0]=-100

    return  dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding['input_ids'].flatten(),#converting from (1,src_max_len) to (src_max_len)
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()

    )
        
      

In [ ]:
#df_trial=df.iloc[:200,:]
#print(df_trial.shape)
er)

(2582, 5)


In [ ]:
for data in sample_dataset:
  print(data)
  break

{'question': 'What is the inheritance pattern of Li–Fraumeni syndrome?', 'context': 'Balanced t(11;15)(q23;q15) in a TP53+/+ breast cancer patient from a Li-Fraumeni syndrome family. Li-Fraumeni Syndrome (LFS) is characterized by early-onset carcinogenesis involving multiple tumor types and shows autosomal dominant inheritance. Approximately 70% of LFS cases are due to germline mutations in the TP53 gene on chromosome 17p13.1. Mutations have also been found in the CHEK2 gene on chromosome 22q11, and others have been mapped to chromosome 11q23. While characterizing an LFS family with a documented defect in TP53, we found one family member who developed bilateral breast cancer at age 37 yet was homozygous for wild-type TP53. Her mother also developed early-onset primary bilateral breast cancer, and a sister had unilateral breast cancer and a soft tissue sarcoma. Cytogenetic analysis using fluorescence in situ hybridization of a primary skin fibroblast cell line revealed that the patient 

In [ ]:
train_df,val_df=train_test_split(df,test_size=0.05)

In [ ]:
train_df.shape,val_df.shape

((2452, 5), (130, 5))

In [ ]:
val_df.iloc[25]

question                Which protein does empagliflozin inhibit?
context         Effect of food on the pharmacokinetics of empa...
answer_text                                                 SGLT2
answer_start                                                   91
answer_end                                                     96
Name: 1665, dtype: object

In [ ]:
class BioQADataModule(pl.LightningDataModule):#this class will help to load our data in efficient manner
  def __init__(self,train_df,test_df,tokenizer,batch_size=8,src_max_len=396,tar_max_len=32):
    super().__init__()
    self.train_df=train_df
    self.test_df=test_df
    self.tokenizer=tokenizer
    self.batch_size=batch_size
    self.src_max_len=src_max_len
    self.tar_max_len=tar_max_len
    print(self.batch_size)
  
  def setup(self):
    self.train_dataset=BioASQDataset(self.train_df,self.tokenizer,self.src_max_len,self.tar_max_len)
    self.test_dataset=BioASQDataset(self.test_df,self.tokenizer,self.src_max_len,self.tar_max_len)
  
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=2
    )
  
  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=1,
        num_workers=2
    )
  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=1,
      num_workers=2
  )

In [ ]:
batch_size=8
no_epochs=6
data_module=BioQADataModule(train_df,val_df,tokenizer,batch_size=batch_size)
data_module.setup()

8


In [ ]:
data_module

In [ ]:
class Model(pl.LightningModule):#main model class
  def __init__(self):
    super().__init__()
    self.model=T5ForConditionalGeneration.from_pretrained(model_name,return_dict=True)
  def forward(self,input_ids,attention_mask,labels=None):
    output=self.model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)

    return output.loss,output.logits #logits shape (batch_size,target_seq_len,vocab_size)
  
  def training_step(self,batch,batch_idx):
    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']
    loss,output=self(input_ids,attention_mask,labels) #use self because calling forward within the same class so we can we use instance self.
    self.log('train_loss',loss,prog_bar=True,logger=True)

    return loss
  def validation_step(self,batch,batch_idx):
    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']
    loss,output=self(input_ids,attention_mask,labels) #use self because calling forward within the same class so we can we use instance self.
    self.log('val_loss',loss,prog_bar=True,logger=True)

    return loss
  def test_step(self,batch,batch_idx):
    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']
    loss,output=self(input_ids,attention_mask,labels) #use self because calling forward within the same class so we can we use instance self.
    self.log('test_loss',loss,prog_bar=True,logger=True)

    return loss
  
  def configure_optimizers(self):
    return AdamW(self.parameters(),lr=0.0001)

  

In [ ]:
model=Model()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath=r"/content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/checkpoints/",
    filename="QA-t5",
    save_top_k=1,
    mode="min",
    verbose=True
)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory /content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/checkpoints/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [ ]:
checkpoint_callback

In [ ]:
trainer=pl.Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=no_epochs,#we always have to pass number of epochs more than the last saved epoch number for ex if last checkpoint created at epoch 4
    #then we have to pass a number more than 4  in max_epochs to resume our training
    gpus=1,
    progress_bar_refresh_rate=30,
    #resume_from_checkpoint = r"/content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/checkpoints/QA-t5.ckpt"
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
#trainer for retraining from checkpoint
# checkpoint_callback_retrain = ModelCheckpoint(
#     monitor="val_loss",
#     dirpath=r"/content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/checkpoints2/",
#     filename="QA-t5",
#     save_top_k=1,
#     mode="min",
#     verbose=True
# )
# trainer_retrain=pl.Trainer(
#     callbacks=[checkpoint_callback_retrain],
#     max_epochs=9,#we always have to pass number of epochs more than the last saved epoch number for ex if last checkpoint created at epoch 4
#     #then we have to pass a number more than 4  in max_epochs to resume our training
#     gpus=1,
#     progress_bar_refresh_rate=30,
#     resume_from_checkpoint = r"/content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/checkpoints/QA-t5.ckpt"
# )

In [ ]:
#retraining starts herer
trainer.fit(model,data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch 3, global step 1227: val_loss reached 0.22466 (best 0.22466), saving model to "/content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/checkpoints/QA-t5.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 4, global step 1534: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 5, global step 1841: val_loss was not in top 1


In [ ]:
#code for clearing gpu's cache 
# import torch
# torch.cuda.memory_summary(device=None, abbreviated=False)
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
trainer.fit(model,data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
#trainer_retrain.test()

In [ ]:
trained_model=Model.load_from_checkpoint(r'/content/drive/MyDrive/Colab Notebooks/QuestionAnsweringDataset/checkpoints/QA-t5.ckpt')

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trained_model.freeze() #freezing our model from training or we can say setting require_grad false as we are doing testing here 
#trained_model.eval() we can use this as well

In [ ]:

def generate_answer(sample_question):#this function will generate the result 
  source_encoding=tokenizer(
  sample_question['question'],
  sample_question['context'],
  max_length=396,
  padding='max_length',
  truncation='only_second',#truncate only the context not the question,
  return_attention_mask=True,
  add_special_tokens=True,
  return_tensors='pt'
  )

  generated_ids=trained_model.model.generate(
      input_ids=source_encoding['input_ids'],
      attention_mask=source_encoding['attention_mask'],
      num_beams=1,
      max_length=80,
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True,
      #use_cache=True
  )

  #print('predicted ids',generated_ids)
  pred=[
        tokenizer.decode(generated_id,skip_special_tokens=True,clean_up_tokenization_spaces=True) for generated_id in generated_ids
  ]

  return " ".join(pred), source_encoding

In [ ]:
for i in range(5):#running on some samples 
  sample_qu=val_df.iloc[i]#12
  print('original---------------')
  print(sample_qu['question'])
  #print(sample_qu['context'])
  print('original answer is ',sample_qu['answer_text'])

  print('model prediction------------------------')
  answer,source_encoding = generate_answer(sample_qu)
  #print('source_encoding',source_encoding['input_ids'])
  print('predicted answer',answer)


  print('----------------------------------------------------')

  # from transformers import T5Tokenizer, T5ForConditionalGeneration

  # tokenizer = T5Tokenizer.from_pretrained('t5-small')
  # model = T5ForConditionalGeneration.from_pretrained('t5-small')


original---------------
What is the mode of inheritance of Romano Ward long QT syndrome?
original answer is  autosomal dominant
model prediction------------------------
predicted answer autosomal dominant
----------------------------------------------------
original---------------
Which enzyme is involved in the maintenance of DNA (cytosine-5-)-methylation?
original answer is  DNMT1
model prediction------------------------
predicted answer DNMT1
----------------------------------------------------
original---------------
Mutation of which gene is associated with McLeod syndrome?
original answer is  XK
model prediction------------------------
predicted answer XK
----------------------------------------------------
original---------------
The small molecule SEA0400 is an inhibitor of which ion antiporter/exchanger?
original answer is  NCX
model prediction------------------------
predicted answer Na+-Ca2
----------------------------------------------------
original---------------
What tyr